In [2]:
# აუცილებელი ბიბლიოთეკები
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1. მონაცემების ჩატვირთვა
train = pd.read_csv("train.csv")  # გადმოწერეთ Kaggle-დან და დადეთ სამუშაო ფოლდერში
test = pd.read_csv("test.csv")

# 1.1 სწრაფი კვლევა
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("\nTrain dtypes:\n", train.dtypes.value_counts())
display(train.head())

# 1.2 დუბლიკატები და missing values
# წაშალეთ ორმაგი ჩანაწერები (თუ არსებობს)
dups = train.duplicated().sum()
print(f"\nDuplicates in train: {dups}")
if dups > 0:
    train = train.drop_duplicates().reset_index(drop=True)
    print("Dropped duplicates. New shape:", train.shape)

# Missing values — თხრობა
miss_train = train.isnull().sum().sort_values(ascending=False)
miss_train = miss_train[miss_train > 0]
print("\nMissing values in train (top):")
print(miss_train.head(30))

miss_test = test.isnull().sum().sort_values(ascending=False)
miss_test = miss_test[miss_test > 0]
print("\nMissing values in test (top):")
print(miss_test.head(30))

# 1.3 - კატეგორიული vs რიცხვითი ცვლადები
numeric_cols = train.select_dtypes(include=[np.number]).columns.tolist()
# გამორიცხეთ id და target-ს
numeric_cols = [c for c in numeric_cols if c not in ("Id","SalePrice")]
cat_cols = train.select_dtypes(exclude=[np.number]).columns.tolist()
print(f"\nNumeric columns: {len(numeric_cols)} | Categorical columns: {len(cat_cols)}")

# 1.2 (გაგრძელება) — სტრატეგია:
# - რიცხვითი missing: median
# - კატეგორიული missing: ჩავსვათ 'Missing' (სხვა სტრატეგია შეიძლება იყოს mode ან სპეციალური კატეგორია)
# (შეგიძლიათ კონკრეტულ სვეტებზე სხვა მიდგომა მიიღოთ, მაგრამ ქვემოთ ზოგადი მიდგომაა)

num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="constant", fill_value="Missing")

# 1.3 Encoding: OneHot dla categorical (handle_unknown='ignore') და StandardScaler რიცხვულზე
numeric_transformer = Pipeline(steps=[
    ('imputer', num_imputer),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', cat_imputer),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, cat_cols)
    ], sparse_threshold=0
)

# 1.4 ვამზადებთ X და y
X = train.drop(columns=["Id","SalePrice"])
y = train["SalePrice"]
X_test = test.drop(columns=["Id"]).copy()
test_ids = test["Id"]

# გაყოფა validation–ისთვის
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. მოდელების ჩამოყალიბება — შევქმნით pipeline ყველა მოდელისთვის
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(random_state=42),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
    # დესპანირებისთვის შეგიძლიათ ჩასვათ Lasso, SVR და სხვ.
}

# ფუნქცია: სწავლება, პროგნოზი და შეფასება
def fit_predict_evaluate(name, model, preprocessor, X_train, y_train, X_val, y_val):
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_val)
    mae = mean_absolute_error(y_val, preds)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, preds)
    print(f"\nModel: {name}")
    print(f"MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}")
    return pipe, {"MAE": mae, "MSE": mse, "RMSE": rmse, "R2": r2}

# შევსება results
results = {}
models_pipelines = {}

for name, mdl in models.items():
    pipe, metrics = fit_predict_evaluate(name, mdl, preprocessor, X_train, y_train, X_val, y_val)
    results[name] = metrics
    models_pipelines[name] = pipe

# 3. შეფასების ცხრილი
results_df = pd.DataFrame(results).T
results_df = results_df[["MAE","MSE","RMSE","R2"]]
print("\nSummary of models:\n")
display(results_df.sort_values("RMSE"))

# 3. დამატებითი: ბოლოდროინდელი საუკეთესო მოდელით პროგნოზი test.csv-ზე და შენახვა
# აქ ვირჩევთ საუკეთესო-მთლიანად RMSE მიხედვით:
best_model_name = results_df["RMSE"].idxmin()
print(f"\nBest model by RMSE: {best_model_name}")
best_pipeline = models_pipelines[best_model_name]

# ძველი test მონაცემების გადასამუშავებლად და პროგნოზისთვის:
test_preds = best_pipeline.predict(X_test)

submission = pd.DataFrame({"Id": test_ids, "SalePrice": test_preds})
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv with predictions from:", best_model_name)


Train shape: (1460, 81)
Test shape: (1459, 80)

Train dtypes:
 object     43
int64      35
float64     3
Name: count, dtype: int64


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000



Duplicates in train: 0

Missing values in train (top):
PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
dtype: int64

Missing values in test (top):
PoolQC          1456
MiscFeature     1408
Alley           1352
Fence           1169
MasVnrType       894
FireplaceQu      730
LotFrontage      227
GarageYrBlt       78
GarageQual        78
GarageFinish      78
GarageCond        78
GarageType        76
BsmtCond          45
BsmtQual          44
BsmtExposure      44
BsmtFinType1      42
BsmtFinType2      42
MasVnrArea        15
MSZoning           4
BsmtHalfBath       2
Utilities          2
Functional         2
BsmtFullBath       2
BsmtFinSF1        

,MAE,MSE,RMSE,R2
GradientBoosting,1.708376e+04,7.454708e+08,2.730331e+04,9.028111e-01
RandomForest,1.779311e+04,8.509722e+08,2.917143e+04,8.890566e-01
Ridge,1.912927e+04,8.836457e+08,2.972618e+04,8.847969e-01
DecisionTree,2.741479e+04,1.745853e+09,4.178341e+04,7.723887e-01
LinearRegression,4.844111e+11,2.547869e+25,5.047642e+12,-3.321722e+15



Best model by RMSE: GradientBoosting
Saved submission.csv with predictions from: GradientBoosting
